In [2]:
import sys
sys.path.append('../')
import os
os.chdir('../')
%reload_ext autoreload
%autoreload 2
os.environ["CUDA_VISIBLE_DEVICES"]="5,6,7"

import os
import json

import torch
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

import models.vqvae as vqvae
import utils.losses as losses 
import options.option_vq as option_vq
import utils.utils_model as utils_model
from dataset import dataset_VQ, dataset_TM_eval
import utils.eval_trans as eval_trans
from options.get_eval_option import get_opt
from models.evaluator_wrapper import EvaluatorModelWrapper
import warnings
warnings.filterwarnings('ignore')
from utils.word_vectorizer import WordVectorizer

In [6]:
class Temp:
    def __init__(self):
        print('mock:: opt')
args = Temp()
args.dataname = args.dataset_name = 't2m'
w_vectorizer = WordVectorizer('./glove', 'our_vab')

if args.dataname == 'kit' : 
    dataset_opt_path = './checkpoints/kit/Comp_v6_KLD005/opt.txt'  
    args.nb_joints = 21
else :
    dataset_opt_path = './checkpoints/t2m/Comp_v6_KLD005/opt.txt'
    args.nb_joints = 22
wrapper_opt = get_opt(dataset_opt_path, torch.device('cuda'))
eval_wrapper = EvaluatorModelWrapper(wrapper_opt)

mock:: opt
Reading ./checkpoints/t2m/Comp_v6_KLD005/opt.txt
Loading Evaluation Model Wrapper (Epoch 28) Completed!!


In [17]:
args.batch_size = 256
args.down_t = 2
args.window_size = 64
train_loader = dataset_VQ.DATALoader(args.dataname,
                                        args.batch_size,
                                        window_size=args.window_size,
                                        unit_length=2**args.down_t)
train_loader_iter = dataset_VQ.cycle(train_loader)

  0%|          | 0/23384 [00:00<?, ?it/s]

100%|██████████| 23384/23384 [00:10<00:00, 2180.04it/s]


Total number of motions 20942


In [7]:
args.nb_code = 8192 #512 # 4096 # 
args.code_dim = 32 #512 # 512 # 
args.output_emb_width = 512
args.down_t = 2
args.stride_t = 2
args.width = 512
args.depth = 3
args.dilation_growth_rate = 3
args.vq_act = 'relu'
args.vq_norm = None

args.quantizer = 'ema_reset'
args.mu = 0.99
net = vqvae.HumanVQVAE(args, ## use args to define different parameters in different quantizers
                       args.nb_code,
                       args.code_dim,
                       args.output_emb_width,
                       args.down_t,
                       args.stride_t,
                       args.width,
                       args.depth,
                       args.dilation_growth_rate,
                       args.vq_act,
                       args.vq_norm)
# args.resume_pth = '/home/epinyoan/git/MaskText2Motion/T2M-BD/output/vq/11_VQVAE_1token_4096_trans_2lyr/net_last.pth'
args.resume_pth = '/home/epinyoan/git/MaskText2Motion/T2M-BD/output/vq/2023-07-19-04-17-17_12_VQVAE_20batchResetNRandom_8192_32/net_last.pth'
ckpt = torch.load(args.resume_pth, map_location='cpu')
# net.load_state_dict(ckpt['net'], strict=True)
# net = torch.nn.DataParallel(net)
net.train()
net.cuda()

args.recons_loss = 'l1_smooth'
args.nb_joints
Loss = losses.ReConsLoss(args.recons_loss, args.nb_joints)

input_emb_width: 263 output_emb_width: 32 down_t: 2 stride_t: 2
width: 512 depth: 3 dilation_growth_rate: 3 activation: relu norm: None
stride_t: 4 2 1
dilation: 1
dilation: 3
dilation: 9
dilation: 1
dilation: 3
dilation: 9
dilation: 1
dilation: 3
dilation: 9
dilation: 1
dilation: 3
dilation: 9


In [14]:
# input_emb_width: 263 output_emb_width: 32 down_t: 2 stride_t: 2
# width: 512 depth: 3 dilation_growth_rate: 3 activation: relu norm: None
class DepthwiseSeparableConv(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding, dilation=1):
        super().__init__()
        self.depthwise = torch.nn.Conv1d(
            in_channels, in_channels, kernel_size=kernel_size, padding=padding, dilation=dilation, groups=in_channels)
        self.pointwise = torch.nn.Conv1d(
            in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out

In [15]:
#### DEBUG 
conv = torch.nn.Conv1d(in_channels=251, out_channels=512, 
                       kernel_size=3, 
                       stride=1, 
                       padding=0)
conv = DepthwiseSeparableConv(in_channels=251, out_channels=512, 
                       kernel_size=3, padding=2, dilation=2)
conv(torch.rand(2, 251, 64)).shape


torch.Size([2, 512, 64])

In [16]:
gt_motion = next(train_loader_iter)
gt_motion.shape

torch.Size([256, 64, 263])

In [5]:
padding = 0
dilation = 1
n_in = n_state = 512
conv1 = torch.nn.Conv1d(n_in, n_state, 3, 1, padding, dilation)
x = torch.rand(256, 512, 64)
conv1(x).shape

torch.Size([256, 512, 62])

In [19]:
args.commit = 0.02
args.loss_vel = 0.5

gt_motion = next(train_loader_iter)
gt_motion = gt_motion.cuda().float() # (bs, 64, dim)

pred_motion, loss_commit, perplexity = net(gt_motion)
loss_motion = Loss(pred_motion, gt_motion)
loss_vel = Loss.forward_vel(pred_motion, gt_motion)

loss = loss_motion + args.commit * loss_commit + args.loss_vel * loss_vel

In [9]:
loss_motion, loss_commit, loss_vel

(tensor(0.8159, device='cuda:0', grad_fn=<SmoothL1LossBackward>),
 tensor(0.0512, device='cuda:0', grad_fn=<MseLossBackward>),
 tensor(0.3440, device='cuda:0', grad_fn=<SmoothL1LossBackward>))

### Distance Codebook

In [9]:
codebook = net.vqvae.quantizer.codebook

x = codebook
k_w = codebook.t()
distance = torch.sum(x ** 2, dim=-1, keepdim=True) - 2 * torch.matmul(x, k_w) + torch.sum(k_w ** 2, dim=0, keepdim=True)
distance = torch.sqrt(distance.clamp(min=1e-8))
distance.max(-1).values.mean(), distance.min(-1).values.mean()
# (distance[0] < 1).sum()
(distance[torch.eye(distance.shape[0]) != 1] < 5).sum()

# rm self position
d_rm_self = distance[torch.eye(distance.shape[0]) != 1].reshape(8192, -1)
d_rm_self.max(-1).values.mean(), d_rm_self.min(-1).values.mean()
# d_rm_self.std(-1).mean()

(tensor(131.3641, device='cuda:0'), tensor(0.0133, device='cuda:0'))

### VQVAE Upper Lower


In [38]:
from models.vqvae_sep import VQVAE_SEP
vqvae_sep = VQVAE_SEP(args, ## use args to define different parameters in different quantizers
                args.nb_code,
                args.code_dim,
                args.output_emb_width,
                args.down_t,
                args.stride_t,
                args.width,
                args.depth,
                args.dilation_growth_rate,
                args.vq_act,
                args.vq_norm)
args.resume_pth = f'/home/epinyoan/git/MaskText2Motion/T2M-BD/output/vq/2023-07-02-03-11-31_16_VQVAE_upperlower_notShareCB/net_last.pth'
ckpt = torch.load(args.resume_pth, map_location='cpu')
# vqvae_sep.load_state_dict(ckpt['net'], strict=True)

vqvae_sep.train()
vqvae_sep.cuda()
''
# gt_motion = next(train_loader_iter)
# gt_motion = gt_motion.cuda().float() # (bs, 64, dim)
# pred_motion, loss_commit, perplexity = vqvae_sep(gt_motion)


input_emb_width: 156 output_emb_width: 16 down_t: 2 stride_t: 2
width: 512 depth: 3 dilation_growth_rate: 3 activation: relu norm: None
stride_t: 4 2 1
input_emb_width: 107 output_emb_width: 16 down_t: 2 stride_t: 2
width: 512 depth: 3 dilation_growth_rate: 3 activation: relu norm: None
stride_t: 4 2 1


''

In [14]:
from models.vqvae_sep import upper_lower_sep
upper_emb, lower_emb = upper_lower_sep(gt_motion)

upper_emb.shape, lower_emb.shape
# [0,1,2,4,5,7,8,10,11] # 9
# [3,6,9,12,13,14,15,16,17,18,19,20,21] # 13
# 89+130

# 4+21*3 + 21*6 + 22*3 + 4
# 4 + 8*3 + 8*6 + 9*3 + 4, 
# 107/263*512

(torch.Size([256, 196, 156]), torch.Size([256, 196, 107]))

In [42]:
ids = vqvae_sep(gt_motion, type='encode')
x_out = vqvae_sep(ids, type='decode')
pred_motion, loss_commit, perplexity = vqvae_sep(gt_motion)

In [12]:
from dataset import dataset_tokenize
import numpy as np
from os.path import join as pjoin
from tqdm import tqdm
train_loader_token = dataset_tokenize.DATALoader(args.dataname, 1, unit_length=2**args.down_t)

  4%|▎         | 845/23384 [00:00<00:05, 4229.16it/s]

100%|██████████| 23384/23384 [00:06<00:00, 3437.35it/s]


In [12]:
codebook_dir = '/home/epinyoan/git/MaskText2Motion/T2M-BD/output/vq/2023-07-02-03-11-31_16_VQVAE_upperlower_notShareCB'
codebook_dir = f'{codebook_dir}/codebook/'
os.makedirs(codebook_dir, exist_ok = True)
for batch in tqdm(train_loader_token):
    pose, name = batch
    bs, seq = pose.shape[0], pose.shape[1]

    pose = pose.cuda().float() # bs, nb_joints, joints_dim, seq_len
    target = vqvae_sep(pose, type='encode')
    target = target.cpu().numpy()
    # np.save(pjoin(codebook_dir, name[0] +'.npy'), target)

  0%|          | 0/22326 [00:00<?, ?it/s]

100%|██████████| 22326/22326 [02:31<00:00, 147.45it/s]


### Train Dataset
- utils/losses.py: reduction = 'none'
- models/vqvae.py:
  - add m_length to vqvae
  - add src_mask_token to encoder/decoder 
- quantize_cnn.py
  - commit_loss = F.mse_loss(x, x_d.detach(), reduction = 'none')

In [12]:
from dataset import dataset_tokenize
train_loader = dataset_tokenize.DATALoader(args.dataname, args.batch_size, unit_length=2**args.down_t)
train_loader_iter = dataset_VQ.cycle(train_loader)

  0%|          | 0/23384 [00:00<?, ?it/s]

100%|██████████| 23384/23384 [00:08<00:00, 2715.30it/s]


In [ ]:
from exit.utils import get_model, generate_src_mask
args.commit = 0.02
args.loss_vel = 0.5

gt_motion, m_length = next(iter(train_loader))
m_length = m_length.cuda()
gt_motion = gt_motion.cuda().float() # (bs, 64, dim)

pred_motion, loss_commit, perplexity = net(gt_motion, m_length)
loss_motion = Loss(pred_motion, gt_motion)
loss_vel = Loss.forward_vel(pred_motion, gt_motion)

# loss = loss_motion + args.commit * loss_commit + args.loss_vel * loss_vel

In [ ]:
# Commit loss
loss_commit = loss_commit.view(gt_motion.shape[0], int(gt_motion.shape[1]/4), -1)
src_mask_token = generate_src_mask(int(gt_motion.shape[1]/4), (m_length/4).int())
loss_commit_seq_masked = torch.masked_select(loss_commit, src_mask_token.unsqueeze(-1)).view(-1, loss_commit.shape[-1]).mean(-1)

weights_token = src_mask_token / (src_mask_token.sum(-1).unsqueeze(-1) * src_mask_token.shape[0])
weight_seq_masked_token = torch.masked_select(weights_token, src_mask_token)

loss_commit_seq_masked = (loss_commit_seq_masked * weight_seq_masked_token).sum()
loss_commit_seq_masked

tensor(0.0715, device='cuda:0', grad_fn=<SumBackward0>)

In [8]:
# loss motion & loss vel
src_mask = generate_src_mask(gt_motion.shape[1], m_length)
weights = src_mask / (src_mask.sum(-1).unsqueeze(-1) * src_mask.shape[0])
weight_seq_masked = torch.masked_select(weights, src_mask)

loss_motion = Loss(pred_motion, gt_motion)
loss_motion_seq_masked = torch.masked_select(loss_motion, src_mask.unsqueeze(-1)).view(-1, loss_motion.shape[-1]).mean(-1)

loss_vel = Loss.forward_vel(pred_motion, gt_motion)
loss_vel_seq_masked = torch.masked_select(loss_vel, src_mask.unsqueeze(-1)).view(-1, loss_vel.shape[-1]).mean(-1)


loss_motion_seq_masked = (loss_motion_seq_masked * weight_seq_masked).sum()
loss_vel_seq_masked = (loss_vel_seq_masked * weight_seq_masked).sum()
loss_motion_seq_masked, loss_vel_seq_masked

(tensor(0.8737, device='cuda:0', grad_fn=<SumBackward0>),
 tensor(0.4324, device='cuda:0', grad_fn=<SumBackward0>))

In [9]:
loss = loss_motion_seq_masked + args.commit * loss_commit_seq_masked + args.loss_vel * loss_vel_seq_masked

### Visualize reconstruction

In [9]:
from exit.utils import get_model, visualize_2motions
import numpy as np
from os.path import join as pjoin
if args.dataname == 't2m':
    data_root = './dataset/HumanML3D'
elif args.dataname == 'kit':
    data_root = './dataset/HumanML3D'

meta_dir = 'checkpoints/t2m/VQVAEV3_CB1024_CMT_H1024_NRES3/meta'
mean = np.load(pjoin(meta_dir, 'mean.npy'))
std = np.load(pjoin(meta_dir, 'std.npy'))

In [18]:
k = 2
pred_motion, loss_commit, perplexity = net(gt_motion)
x = gt_motion[k].detach().cpu().numpy()
y = pred_motion[k].detach().cpu().numpy()
visualize_2motions(x, std, mean, args.dataname, x.shape[0], y, save_path=None)

### Eval

In [6]:
val_loader = dataset_TM_eval.DATALoader(args.dataname, False,
                                        32,
                                        w_vectorizer,
                                        unit_length=2**args.down_t)

In [7]:
class LoggerWriterMock:
    def __init__(self):
        self.info
    def info(self, *args):
        print(*args)
    def add_scalar(self, *args):
        print(*args)
logger = LoggerWriterMock()
logger.info('test')
writer = LoggerWriterMock()
writer.add_scalar('./Test/FID', 'fid', 'nb_iter')

In [ ]:
args.out_dir = './'
a = eval_trans.evaluation_vqvae(
    args.out_dir, 
    val_loader, 
    ################################################################
    net, #vqvae_sep, # net <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    ################################################################
    logger, 
    writer, 
    nb_iter = 1, 
    best_fid = 1, 
    best_iter = 1, 
    best_div = 1, 
    best_top1 = 1, 
    best_top2 = 1, 
    best_top3 = 1, 
    best_matching = 1, 
    eval_wrapper=eval_wrapper)


In [ ]:
word_embeddings, pos_one_hots, caption, sent_len, motion, m_length, token, name = next(iter(val_loader))
motion.shape